In [3]:
!pip install qiskit
!pip install qiskit_aer


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 80.1 MB/s eta 0:00:00


In [7]:
# Import necessary libraries from Qiskit
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from qiskit.circuit.library import QFTGate # Updated import
import matplotlib.pyplot as plt
import numpy as np

# Define a helper function for controlled modular exponentiation
# This is a crucial part of Shor's algorithm, implementing the U_f operation.
# It applies the transformation |x>|y> -> |x>|y + a^x mod N>
def controlled_mod_exp(qc, control_qubit, target_qubits, a, N, aux_qubits_start, num_aux_qubits):
    """
    Implements controlled modular exponentiation (a^x mod N).
    This is a placeholder for a more complex circuit that would perform
    the actual modular exponentiation. For a simplified example like factoring 15,
    this part is often hardcoded or simplified for demonstration.

    Args:
        qc (QuantumCircuit): The quantum circuit to apply operations to.
        control_qubit (int): The index of the control qubit.
        target_qubits (list): A list of indices for the target qubits (for x).
        a (int): The base of the exponentiation.
        N (int): The modulus.
        aux_qubits_start (int): Starting index for auxiliary qubits if needed.
        num_aux_qubits (int): Number of auxiliary qubits if needed.
    """
    # In a real Shor's algorithm, this would involve a complex circuit
    # implementing modular exponentiation. For this simplified example,
    # we're just applying a simple CNOT for demonstration purposes,
    # as the '0000' output suggests a very basic operation not truly factoring.
    # To factor 15, 'a' would typically be coprime to 15 (e.g., 7 or 11).
    # The actual modular exponentiation circuit is highly non-trivial.
    qc.cx(control_qubit, target_qubits[0]) # A placeholder operation.
                                          # This specific CNOT does not perform
                                          # the full modular exponentiation for Shor's.
                                          # It's a simplification for a basic circuit.

# Define the number to factor (N)
N = 15
# Define a coprime number (a) - typically chosen randomly,
# but for a simplified example, a fixed value is used.
# For N=15, a=7 is a common choice as gcd(7,15)=1.
a = 7

# Create a quantum circuit with 4 qubits and 4 classical bits
# Qubits:
# - 3 counting qubits (for phase estimation, typically more for larger N)
# - 1 target qubit (for modular exponentiation result, typically more for larger N)
qc = QuantumCircuit(4, 4)

# Initialize the target qubit to |1>
# This is common in Shor's algorithm setups.
qc.x(3) # Applies an X-gate to qubit 3, changing its state from |0> to |1>

# Apply Hadamard gates to the counting qubits
# This creates a superposition of all possible states on these qubits,
# which is essential for phase estimation.
qc.h([0, 1, 2]) # Applies Hadamard gates to qubits 0, 1, and 2

# Apply controlled modular exponentiation
# This is the core of the quantum part of Shor's algorithm.
# The `controlled_mod_exp` function (simplified here) would apply
# U_a^x |y> = |y + a^x mod N>
# where x is the state of the control qubits.
# The parameters (base, 8, N, 2, 3) in the original code seem to be
# a bit abstract for a direct mapping to 'a' and 'N'.
# For a conceptual example, we'll use a simplified call.
# The original code's `controlled_mod_exp` only uses one control and one target.
# Let's adjust to fit the simplified `controlled_mod_exp` for demonstration.
# In a true Shor's, there would be multiple controlled-U operations,
# one for each counting qubit.
# For this simplified example, the `controlled_mod_exp` is too basic to
# properly factor N=15. The output '0000' confirms this.
# This section needs a more complex U^2^k mod N gate for each control qubit.

# Example of what would be needed for a more complete algorithm (conceptual):
# For each counting qubit k, apply U^(2^k) mod N
# qc.append(controlled_U_mod_N(a, N, 2**0), [0] + target_qubits + aux_qubits)
# qc.append(controlled_U_mod_N(a, N, 2**1), [1] + target_qubits + aux_qubits)
# qc.append(controlled_U_mod_N(a, N, 2**2), [2] + target_qubits + aux_qubits)

# The original code had:
# controlled_mod_exp(qc, base, 8, N, 2, 3)
# Given the provided `controlled_mod_exp` function, it only takes one control and one target.
# To make it runnable and consistent with the provided `controlled_mod_exp` (which is very basic),
# we will apply a single CNOT from qubit 0 to qubit 3 as a conceptual placeholder.
# This does NOT implement Shor's algorithm's modular exponentiation.
qc.cx(0, 3) # This is a highly simplified placeholder.
            # In a real Shor's, this would be a series of
            # controlled-U^2^k operations.

# Apply the inverse Quantum Fourier Transform (QFT) to the counting qubits
# The QFT transforms the phase information (encoded by modular exponentiation)
# into measurable probabilities in the computational basis.
qc.append(QFTGate(3).inverse(), [0, 1, 2]) # Updated to use QFTGate
                                                     # applies QFT without final swaps.
                                                     # .inverse() gets the inverse QFT.
                                                     # Applied to qubits 0, 1, 2.



In [6]:
# Measure the qubits
# The results of these measurements will be used to classically
# determine the period of the function a^x mod N.
qc.measure([0, 1, 2, 3], [0, 1, 2, 3]) # Measure quantum bits 0,1,2,3 into classical bits 0,1,2,3

# Use the Qiskit Aer simulator to run the circuit
simulator = AerSimulator() # Initializes the AerSimulator, a local quantum simulator.
compiled_circuit = transpile(qc, simulator) # Optimizes the circuit for the simulator.
result = simulator.run(compiled_circuit).result() # Runs the simulation and gets the results.

# Get the counts of outcomes
counts = result.get_counts() # Retrieves the measurement outcomes as a dictionary of counts.

print("Counts:", counts) # Prints the measurement counts.

# Visualize the results
plot_histogram(counts) # Generates a histogram of the measurement results.
plt.show() # Displays the histogram.

qc.draw('text') # Draws the quantum circuit using text.

Counts: {'0000': 252, '0100': 250, '1100': 251, '1000': 271}


┌───┐     ┌─────────┐┌─┐         ┌─┐      
q_0: ┤ H ├──■──┤0        ├┤M├─────────┤M├──────
     ├───┤  │  │         │└╥┘┌─┐      └╥┘┌─┐   
q_1: ┤ H ├──┼──┤1 qft_dg ├─╫─┤M├───────╫─┤M├───
     ├───┤  │  │         │ ║ └╥┘┌─┐    ║ └╥┘┌─┐
q_2: ┤ H ├──┼──┤2        ├─╫──╫─┤M├────╫──╫─┤M├
     ├───┤┌─┴─┐└───┬─┬───┘ ║  ║ └╥┘┌─┐ ║  ║ └╥┘
q_3: ┤ X ├┤ X ├────┤M├─────╫──╫──╫─┤M├─╫──╫──╫─
     └───┘└───┘    └╥┘     ║  ║  ║ └╥┘ ║  ║  ║ 
c: 4/═══════════════╩══════╩══╩══╩══╩══╩══╩══╩═
                    3      0  1  2  3  0  1  2